In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 4,
    "Parts": 5,
    "Warehouse": 6,
    "Supplier": 7,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "test17"

In [4]:
create_ops, graph = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

from networkx.readwrite import json_graph
print(json_graph.node_link_data(graph[0], edges="links"))

direct_create_payload = {
    "version": version,
    "action": "direct_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": json_graph.node_link_data(graph[0], edges="links")
}

requests.post(f"{API_URL}/schema/live/update", json=direct_create_payload)


Generated 681 operations
{'directed': True, 'multigraph': False, 'graph': {}, 'nodes': [{'id': '1', 'name': 'Name_3597', 'description': 'Description_6772', 'type': 'Type A', 'cost': 8765.73, 'importance': 3, 'expected_life': 421, 'units_in_chain': 32, 'expiry': 274, 'node_type': 'Parts', 'usage': 'core'}, {'id': '2', 'name': 'Name_7545', 'description': 'Description_3681', 'type': 'Type A', 'cost': 9453.46, 'importance': 1, 'expected_life': 2276, 'units_in_chain': 76, 'expiry': 289, 'node_type': 'Parts', 'usage': 'core'}, {'id': '3', 'name': 'Name_5104', 'description': 'Description_5361', 'type': 'Type C', 'cost': 1826.22, 'importance': 3, 'expected_life': 971, 'units_in_chain': 35, 'expiry': 211, 'node_type': 'Parts', 'usage': 'core'}, {'id': '4', 'name': 'Name_1750', 'description': 'Description_7132', 'type': 'Type C', 'cost': 7163.83, 'importance': 2, 'expected_life': 762, 'units_in_chain': 74, 'expiry': 274, 'node_type': 'Parts', 'usage': 'core'}, {'id': '5', 'name': 'Name_4803', 'd

<Response [200]>

In [5]:
# bulk_create_payload = {
#     "version": version,
#     "action": "bulk_create",
#     "type": "schema",
#     "timestamp": create_ops[0]["timestamp"],
#     "payload": []
# }

# for op in create_ops:
#     bulk_create_payload["payload"].append(op["payload"])

# print(bulk_create_payload)

# requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

In [6]:
update_cycles = 5
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

Generated 17 update operations
{'version': 'test17', 'action': 'bulk_update', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': 'Supplier_3', 'node_type': 'Supplier', 'payload': {'id': 'Supplier_3', 'name': 'Name_1380', 'location': 'Chicago', 'quality': 783.75, 'reliability': 0.71, 'size': 'Extra Large'}}, {'node_id': '3-3-1', 'node_type': 'ProductOffering', 'payload': {'id': '3-3-1', 'name': 'Name_5940', 'cost': 5024.21, 'demand': 888}}, {'node_id': '5', 'node_type': 'Parts', 'payload': {'id': '5', 'name': 'Name_7500', 'description': 'Description_3782', 'type': 'Type C', 'cost': 3888.36, 'importance': 4, 'expected_life': 1507, 'units_in_chain': 76, 'expiry': 76}}, {'node_id': '5-1-3', 'node_type': 'ProductOffering', 'payload': {'id': '5-1-3', 'name': 'Name_1270', 'cost': 2595.23, 'demand': 125}}, {'node_id': '5-4-3', 'node_type': 'ProductOffering', 'payload': {'id': '5-4-3', 'name': 'Name_3983', 'cost': 8747.3, 'demand': 346}}, {'node_id': '4-4-2', 'node_type': 'ProductOfferin

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)